In [ ]:
import numpy as np
import pandas as pd
import time 
import random
from sklearn.model_selection import train_test_split
from sklearn import datasets, preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
ds=pd.read_csv('/content/drive/MyDrive/diabetes.csv')
ds.head(15)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


# Import Data

In [ ]:

x = ds.drop(['Outcome'], axis = 1).values
y = ds['Outcome'].values
y = y.reshape(y.shape[0], 1)
#Spliting Data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.25,random_state=0)
#Standardize data
from sklearn.preprocessing import StandardScaler 
scale = StandardScaler()
x_train_std = scale.fit_transform(x_train)
x_test_std = scale.transform(x_test)


In [ ]:
print(y_train[0][0])

0


In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

# Feed Forward

In [ ]:
def FF(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer

In [ ]:
NIL=8
HL=18
OL=1
#144+18+19=181

In [ ]:
def MLP(solution):
      fitness=0
      w=(NIL*HL)+(HL*OL)
      b=(NIL*HL)+HL+(HL*OL)+OL
      W=solution[0:w]
      B=solution[w:b]
      for idx,x in enumerate(x_train_std):
        actualValue=FF(HL,OL,W,B,x)

        fitness+=pow((actualValue[0]-y_train[idx][0]),2)

      fitness=fitness/len(x_train)
      
      return fitness

# BAT Algorithm

In [ ]:

import random
import numpy
import time
import math
import sklearn
from numpy import asarray
from sklearn.preprocessing import normalize

class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual = []
        self.convergence = []
        self.optimizer = ""
        self.objfname = ""
        self.startTime = 0
        self.endTime = 0
        self.executionTime = 0
        self.lb = 0
        self.ub = 0
        self.dim = 0
        self.popnum = 0
        self.maxiers = 0




def BAT(objf, lb, ub, dim, N, Max_iteration):

    n = N
    # Population size

    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim
    N_gen = Max_iteration  # Number of generations

    A = 0.5
    # Loudness  (constant or decreasing)
    r = 0.5
    # Pulse rate (constant or decreasing)

    Qmin = 0  # Frequency minimum
    Qmax = 2  # Frequency maximum

    d = dim  # Number of dimensions

    # Initializing arrays
    Q = numpy.zeros(n)  # Frequency
    v = numpy.zeros((n, d))  # Velocities
    Convergence_curve = []

    # Initialize the population/solutions
    Sol = numpy.zeros((n, d))
    for i in range(dim):
        Sol[:, i] = numpy.random.rand(n) * (ub[i] - lb[i]) + lb[i]

    S = numpy.zeros((n, d))
    S = numpy.copy(Sol)
    Fitness = numpy.zeros(n)

    # initialize solution for the final results
    s = solution()
    print('BAT is optimizing  "' + objf.__name__ + '"')

    # Initialize timer for the experiment
    timerStart = time.time()
    s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")

    # Evaluate initial random solutions
    for i in range(0, n):
        Fitness[i] = objf(Sol[i, :])

    # Find the initial best solution and minimum fitness
    I = numpy.argmin(Fitness)
    best = Sol[I, :]
    fmin = min(Fitness)

    # Main loop
    for t in range(0, N_gen):

        # Loop over all bats(solutions)
        for i in range(0, n):
            Q[i] = Qmin + (Qmin - Qmax) * random.random()
            v[i, :] = v[i, :] + (Sol[i, :] - best) * Q[i]
            S[i, :] = Sol[i, :] + v[i, :]

            # Check boundaries
            for j in range(d):
                Sol[i, j] = numpy.clip(Sol[i, j], lb[j], ub[j])

            # Pulse rate
            if random.random() > r:
                S[i, :] = best + 0.001 * numpy.random.randn(d)

            # Evaluate new solutions
            Fnew = objf(S[i, :])

            # Update if the solution improves
            if (Fnew <= Fitness[i]) and (random.random() < A):
                Sol[i, :] = numpy.copy(S[i, :])
                Fitness[i] = Fnew

            # Update the current best solution
            if Fnew <= fmin:
                best = numpy.copy(S[i, :])
                fmin = Fnew

        # update convergence curve
        Convergence_curve.append(fmin)

        if t % 9 == 0:
            print(["At iteration " + str(t) + " the best fitness is " + str(fmin)])

    timerEnd = time.time()
    s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime = timerEnd - timerStart
    s.convergence = Convergence_curve
    s.optimizer = "BAT"
    s.objfname = objf.__name__
    s.bestIndividual=best

    return s

In [ ]:
A=BAT(MLP, -1, 1, 181, 50, 1000)


BAT is optimizing  "MLP"
['At iteration 0 the best fitness is 0.1843708545773074']
['At iteration 9 the best fitness is 0.17847506449867392']
['At iteration 18 the best fitness is 0.17606009609499776']
['At iteration 27 the best fitness is 0.17405820027390081']
['At iteration 36 the best fitness is 0.17189121553959658']
['At iteration 45 the best fitness is 0.17024427583010723']
['At iteration 54 the best fitness is 0.16893604360625872']
['At iteration 63 the best fitness is 0.16751784625120666']
['At iteration 72 the best fitness is 0.1661618073632037']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


['At iteration 81 the best fitness is 0.16500096471058712']
['At iteration 90 the best fitness is 0.1639630955322925']
['At iteration 99 the best fitness is 0.1629066341450631']
['At iteration 108 the best fitness is 0.16203639413486431']
['At iteration 117 the best fitness is 0.1612483478720979']
['At iteration 126 the best fitness is 0.16052510882684065']
['At iteration 135 the best fitness is 0.15985352656960852']
['At iteration 144 the best fitness is 0.1591807695937505']
['At iteration 153 the best fitness is 0.1585974767702034']
['At iteration 162 the best fitness is 0.157962505260531']
['At iteration 171 the best fitness is 0.15740121807239338']
['At iteration 180 the best fitness is 0.15683628562138896']
['At iteration 189 the best fitness is 0.15629240647746703']
['At iteration 198 the best fitness is 0.15582978436208175']
['At iteration 207 the best fitness is 0.15530378884165427']
['At iteration 216 the best fitness is 0.15488304457032492']
['At iteration 225 the best fitnes

In [ ]:
print(A.convergence[999])

0.12054701732301289


In [ ]:
solution=A.bestIndividual
print(solution)

[ 0.73209084 -0.86502133 -0.57642649  1.22192587 -2.13403999  0.60632047
  0.86677599  1.24562251  1.68633048  0.33977064  0.01784838  1.08568234
  1.39462177 -1.91940331 -1.02246151 -0.49114266 -1.82968507 -0.48231602
  0.7139712  -1.43097148 -0.64526452  1.06753892 -0.97827882  1.00580322
 -1.69449244  2.33256118 -1.94547755 -1.5344631   0.80926255  0.66303229
  0.93581735 -0.16891629 -0.19778    -0.06521807 -1.22037434  2.17008763
 -0.46137378 -1.30636697 -0.5351861   0.81378394  0.76143371 -1.25294025
  0.98417342  0.22037125 -0.49961473 -0.91339945 -1.13859779 -3.26986146
 -0.23400245  0.99044716 -0.56938397  0.39638676  0.91008315 -0.13535668
 -0.77507982  0.92875902  0.64128491  0.80743849 -0.0514977  -0.79472065
  1.73101561  0.37579009 -0.25424833 -2.15173699  0.48437092  0.27865921
  2.36084851  1.05724932  0.06848296  0.81490375  0.44432071  0.79835266
  0.87626606  1.51974739  1.40967643  0.83236752  0.96482453  1.1149408
 -0.90212031  2.69235103  0.87325135 -0.14023001 -0.

In [ ]:
def Test(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer




In [ ]:
min_key=min(B.keys(), key=(lambda k: B[k]))

In [ ]:
w=(NIL*HL)+(HL*OL)
b=(NIL*HL)+HL+(HL*OL)+OL    
W=solution[0:w]
B=solution[w:b]
print(len(W))
print(B)

162
[-1.48913921  0.27189234  1.60753073  1.04156739 -1.41893288  0.28047332
  0.85345053  1.34474772 -0.62464357  0.7217644  -0.51691287 -0.81368186
  0.82397087  0.09214988  0.96654203  1.18645286 -0.46336883  0.84480702
  2.25520901]


In [ ]:
a=Test(HL,OL,W,B,x_test_std[7])
print(a)


[1.]


In [ ]:
R=0
for i in range(len(y_train)):
  p=Test(HL,OL,W,B,x_train_std[i])
  if y_train[i][0]==0:
    if p[0]<0.5:
      R+=1
  if y_train[i][0]==1:
    if p[0]>0.5:
      R+=1


In [ ]:
print((R*100)/len(y_train))

83.85416666666667


In [ ]:
def prediction(HL,OL,W,B,x):
  predict=[]
  s=[]
  for i in range(len(x)):
    a=Test(HL,OL,W,B,x[i])
    if a[0]>0.5:
      predict.append(1)
    else:
      predict.append(0)
    
    
  
  return predict

In [ ]:
pred=prediction(HL,OL,W,B,x_test_std)
print(f"Predict :{pred}")
y_true = sum(y_test.tolist(), [])
print(f"true y:{y_true}")

acc=0
for i in range(len(y_test)):
  if pred[i]==y_true[i]:
    acc+=1

print(f"accuracy:{acc/len(y_test)*100}%")

Predict :[1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
true y:[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1

In [ ]:
p=[[2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173583065964e-08], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859951413703e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999997594005218, 1.7827926890587977e-14], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3777059110728068e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3441849876730426e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [2.3443797239678989e-07, 1.0], [0.999997046667274, 1.0], [3.4230559529451456e-19, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.7353786067445575e-13, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998619, 7.412144614465116e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [3.2958383525020503e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537748567615e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.735378606784197e-13, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859963200897e-07, 1.0], [1.0, 7.49479046619701e-09], [2.730081969202577e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [2.824682922217676e-13, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.494780457447032e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.2482862984116327e-18, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.494790466200846e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.7601836872318313e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999999998, 3.6456980727045307e-10], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [3.561895408985392e-19, 1.0], [0.9999999999998617, 7.413141488936635e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.999997046667274, 1.0], [1.0, 7.49479046619701e-09], [2.3443797226534723e-07, 1.0], [0.9999999999997193, 1.377389407325726e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.735371990190059e-13, 1.0], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [1.0, 5.790537731467655e-12], [0.9999999954925718, 1.1219721194964509e-13], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.735378606754956e-13, 1.0], [2.344379723967903e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.7601861964494985e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 1.4646590413208634e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.416891915703537e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [4.7601859947178284e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997216, 1.382760290185998e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.494787237171255e-09], [0.9999997593963287, 1.7827783200580683e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 5.790537731467655e-12], [5.186223429698876e-06, 1.0], [1.0, 7.49479046619701e-09], [3.2963276408309863e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.760185995141387e-07, 1.0], [0.999999759396326, 1.7827783109247703e-14], [2.3443797239678989e-07, 1.0], [2.3443797239533066e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [3.539949630789169e-14, 1.0], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071096696e-17], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.399724176787439e-09], [3.2958381859094635e-19, 1.0], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.736071297671768e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.494790466196905e-09], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997597973544, 1.784154246233934e-14], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773916445774456e-17], [1.0, 5.7905377175175346e-12], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105640535e-14], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [4.760185994289154e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997594030715, 1.782801427057929e-14], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071148567e-17], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998492, 1.8358616739546913e-17], [2.3443797239678989e-07, 1.0], [2.3443792937421956e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.494790466200632e-09], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [0.9999999999997193, 1.3773894071084169e-17], [0.9999999999997193, 1.3773894071086518e-17], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.8247268646226163e-13, 1.0], [4.760185922590398e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.493736721731287e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995142792e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [3.295837933196582e-19, 1.0], [1.0, 5.7905377314674905e-12], [1.0, 5.790539620623675e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.492433652437726e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [5.0873878446689e-18, 1.0]]
for i in range(len(pred)):
  if p[i][0] and p[i][1] >0.5:
    print(i)
  